## Detailed Article Explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/542132/comparing-gpt-4o-vs-claude-3-5-sonnet-for-zero-shot-text-classification

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Importing and Installing Required Libraries

In [ ]:
!pip install anthropic
!pip install openai

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import anthropic
from openai import OpenAI


C:\Users\usman\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Importing and Preprocessing the Dataset

In [2]:
## Dataset download link
## https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment?select=Tweets.csv

dataset = pd.read_csv(r"D:\Datasets\tweets.csv")
print(dataset.shape)
dataset.head()

(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
# Remove rows where 'airline_sentiment' or 'text' are NaN
dataset = dataset.dropna(subset=['airline_sentiment', 'text'])

# Remove rows where 'airline_sentiment' or 'text' are empty strings
dataset = dataset[(dataset['airline_sentiment'].str.strip() != '') & (dataset['text'].str.strip() != '')]

# Filter the DataFrame for each sentiment
neutral_df = dataset[dataset['airline_sentiment'] == 'neutral']
positive_df = dataset[dataset['airline_sentiment'] == 'positive']
negative_df = dataset[dataset['airline_sentiment'] == 'negative']

# Randomly sample records from each sentiment
neutral_sample = neutral_df.sample(n=34)
positive_sample = positive_df.sample(n=33)
negative_sample = negative_df.sample(n=33)

# Concatenate the samples into one DataFrame
dataset = pd.concat([neutral_sample, positive_sample, negative_sample])

# Reset index if needed
dataset.reset_index(drop=True, inplace=True)

# print value counts
print(dataset["airline_sentiment"].value_counts())

airline_sentiment
neutral     34
positive    33
negative    33
Name: count, dtype: int64


## Zero Shot Text Classification with GPT-4o

In [ ]:
def find_sentiment(client, model):
    
    tweets_list = dataset["text"].tolist()

    
    all_sentiments = []
        
    i = 0
    exceptions = 0
    while i < len(tweets_list):

        try:
            tweet = tweets_list[i]
            content = """What is the sentiment expressed in the following tweet about an airline?
            Select sentiment value from positive, negative, or neutral. Return only the sentiment value in small letters.
            tweet: {}""".format(tweet)
                
            sentiment_value = client.chat.completions.create(
                                  model= model,
                                  temperature = 0,
                                  max_tokens = 10,
                                  messages=[
                                        {"role": "user", "content": content}
                                    ]
                                ).choices[0].message.content

            all_sentiments.append(sentiment_value)
            i = i + 1
            print(i, sentiment_value)

        except Exception as e:
            print("===================")
            print("Exception occurred:", e)
            exceptions += 1

    print("Total exception count:", exceptions)
    accuracy = accuracy_score(all_sentiments, dataset["airline_sentiment"])
    print("Accuracy:", accuracy)

In [19]:
%%time
client = OpenAI(
    # This is the default and can be omitted
    api_key = os.environ.get('OPENAI_API_KEY'),
)
model = "gpt-4o"

find_sentiment(client, model)


1 neutral
2 negative
3 neutral
4 neutral
5 neutral
6 neutral
7 neutral
8 neutral
9 neutral
10 neutral
11 neutral
12 neutral
13 neutral
14 negative
15 positive
16 negative
17 neutral
18 positive
19 negative
20 positive
21 neutral
22 positive
23 neutral
24 neutral
25 neutral
26 negative
27 negative
28 neutral
29 neutral
30 negative
31 neutral
32 negative
33 negative
34 negative
35 positive
36 positive
37 positive
38 positive
39 positive
40 positive
41 positive
42 negative
43 positive
44 positive
45 negative
46 positive
47 positive
48 neutral
49 positive
50 positive
51 positive
52 positive
53 positive
54 negative
55 neutral
56 positive
57 negative
58 positive
59 positive
60 positive
61 positive
62 positive
63 positive
64 neutral
65 positive
66 positive
67 positive
68 negative
69 negative
70 negative
71 negative
72 negative
73 neutral
74 negative
75 negative
76 negative
77 neutral
78 negative
79 negative
80 negative
81 negative
82 negative
83 negative
84 negative
85 negative
86 positive
87

## Zero Shot Text Classification with Claude 3.5-Sonnet

In [21]:
def find_sentiment_claude(client, model):
    
    tweets_list = dataset["text"].tolist()

    
    all_sentiments = []
        
    i = 0
    exceptions = 0
    while i < len(tweets_list):

        try:
            tweet = tweets_list[i]
            content = """What is the sentiment expressed in the following tweet about an airline?
            Select sentiment value from positive, negative, or neutral. Return only the sentiment value in small letters.
            tweet: {}""".format(tweet)
                
            sentiment_value = client.messages.create(
                                model= model,
                                max_tokens=10,
                                temperature=0.0,
                                messages=[
                                    {"role": "user", "content": content}
                                ]
                            ).content[0].text

            all_sentiments.append(sentiment_value)
            i = i + 1
            print(i, sentiment_value)

        except Exception as e:
            print("===================")
            print("Exception occurred:", e)
            exceptions += 1

    print("Total exception count:", exceptions)
    accuracy = accuracy_score(all_sentiments, dataset["airline_sentiment"])
    print("Accuracy:", accuracy)

In [24]:
%%time
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key = os.environ.get('ANTHROPIC_API_KEY')
)

model = "claude-3-5-sonnet-20240620"

find_sentiment_claude(client, model)

1 neutral
2 negative
3 neutral
4 neutral
5 neutral
6 neutral
7 neutral
8 neutral
9 neutral
10 neutral
11 neutral
12 neutral
13 neutral
14 neutral
15 neutral
16 negative
17 neutral
18 neutral
19 negative
20 neutral
21 neutral
22 positive
23 neutral
24 neutral
25 neutral
26 negative
27 negative
28 neutral
29 neutral
30 neutral
31 neutral
32 negative
33 neutral
34 negative
35 positive
36 positive
37 positive
38 positive
39 positive
40 positive
41 positive
42 negative
43 positive
44 positive
45 negative
46 positive
47 positive
48 neutral
49 positive
50 positive
51 positive
52 positive
53 positive
54 positive
55 neutral
56 positive
57 negative
58 positive
59 positive
60 positive
61 positive
62 positive
63 positive
64 positive
65 positive
66 positive
67 positive
68 negative
69 negative
70 negative
71 negative
72 negative
73 neutral
74 negative
75 negative
76 negative
77 neutral
78 negative
79 negative
80 negative
81 negative
82 negative
83 negative
84 negative
85 negative
86 positive
87 nega